In [2]:
pip install numba

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
    --------------------------------------- 0.0/2.7 MB 487.6 kB/s eta 0:00:06
   - -------------------------------------- 0.1/2.7 MB 1.0 MB/s eta 0:00:03
   -------- ------------------------------- 0.6/2.7 MB 3.5 MB/s eta 0:00:01
   -------------- ------------------------- 1.0/2.7 MB 5.2 MB/s eta 0:00:01
   ----------------------------------- ---- 2.4/2.7 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   - -------------------------------------- 1.3/28.1 MB 42.3 MB/s eta 0:00:01
   --- ------------------------------------ 2.2/28.1 MB 28.4 MB/s eta 0:00:01
   ---- ----------------------------------- 3.1/28.1 MB 27.9 MB/s eta 0:00:01
   ------- -------------------------------- 4.9/28.1 MB 28.6 MB/s eta 0:00:01
   --------- -------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\mrun7\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import os
import gc
import heapq
import pickle
import numba as nb
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

C:\Users\mrun7\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
params={
    "ops_weights" : np.array([1.0, 6.0, 3.0]),
    "test_ops_weights" : np.array([1.0, 6.0, 3.0]),
    "time_weight":[3,3,3],
    "hours_co_visitation_click":0.07,
    "hours_co_visitation_buy":0.06,
    "topn":20,
    "seq_weight":[-3,-4,-2], 
    "rank_w":[-4,-4,-5],
    "base_time_weight":[1.4,1.4,1.4],
    "tail":100
}

tail = params["tail"]
parallel = 1024


OP_WEIGHT = 0; TIME_WEIGHT = 1

ops_weights = params["ops_weights"]
test_ops_weights = params["test_ops_weights"]
topn = params["topn"]



time_covisitation=[0,0]
time_covisitation[TIME_WEIGHT]=params["hours_co_visitation_click"]
time_covisitation[OP_WEIGHT]=params["hours_co_visitation_buy"]

In [6]:
df = pd.read_csv(r"C:\Users\mrun7\Downloads\train.csv\train.csv")
df_test = pd.read_csv(r"C:\Users\mrun7\Downloads\test.csv\test.csv")
df = pd.concat([df, df_test]).reset_index(drop = True)
npz = np.load(r"C:\Users\mrun7\Downloads\train.npz\train.npz")
npz_test = np.load(r"C:\Users\mrun7\Downloads\test.npz\test.npz")
aids = np.concatenate([npz['aids'], npz_test['aids']])
ts = np.concatenate([npz['ts'], npz_test['ts']])
ops = np.concatenate([npz['ops'], npz_test['ops']])

df["idx"] = np.cumsum(df.length) - df.length
df["end_time"] = df.start_time + ts[df.idx + df.length - 1]

In [7]:
len_test=len(df_test)

In [8]:
aids.dtype,ts.dtype,ops.dtype

(dtype('int64'), dtype('int64'), dtype('uint8'))

In [9]:
aids[:3],ts[:3],ops[:3]

(array([1517085, 1563459, 1309446]),
 array([    0,   104, 62639]),
 array([0, 0, 0], dtype=uint8))

In [10]:
@nb.jit(nopython = True, cache = True)
def get_single_pairs(pairs, aids, ts, ops, idx, length, start_time, ops_weights, mode,time_weight,hours_co_visitation,base_time_weight):
    max_idx = idx + length
    min_idx = max(max_idx - tail, idx)
    for i in range(min_idx, max_idx):
        for j in range(i + 1, max_idx):
            if ts[j] - ts[i] >= hours_co_visitation * 60 * 60: break
            if aids[i] == aids[j]: continue
            if mode == OP_WEIGHT:
                w1 = ops_weights[ops[j]]
                w2 = ops_weights[ops[i]]
            elif mode == TIME_WEIGHT:
                w1 = base_time_weight[ops[j]] + time_weight[ops[j]] * (ts[i] + start_time - 1659304800) / (1662328791 - 1659304800)
                w2 = base_time_weight[ops[i]] + time_weight[ops[j]] * (ts[j] + start_time - 1659304800) / (1662328791 - 1659304800)
            pairs[(aids[i], aids[j])] = w1
            pairs[(aids[j], aids[i])] = w2

# get pair dict of each session in parallel
# merge pairs into a nested dict format (cnt)
@nb.jit(nopython = True, parallel = True, cache = True)
def get_pairs(aids, ts, ops, row, cnts, ops_weights, mode,time_weight,hours_co_visitation,base_time_weight):
    par_n = len(row)
    pairs = [{(0, 0): 0.0 for _ in range(0)} for _ in range(par_n)]
    for par_i in nb.prange(par_n):
        _, idx, length, start_time = row[par_i]
        get_single_pairs(pairs[par_i], aids, ts, ops, idx, length, start_time, ops_weights, mode,time_weight,hours_co_visitation,base_time_weight)
    for par_i in range(par_n):
        for (aid1, aid2), w in pairs[par_i].items():
            if aid1 not in cnts: cnts[aid1] = {0: 0.0 for _ in range(0)}
            cnt = cnts[aid1]
            if aid2 not in cnt: cnt[aid2] = 0.0
            cnt[aid2] += w
    
# util function to get most common keys from a counter dict using min-heap
# overwrite == 1 means the later item with equal weight is more important
# otherwise, means the former item with equal weight is more important
# the result is ordered from higher weight to lower weight
@nb.jit(nopython = True, cache = True)
def heap_topk(cnt, overwrite, cap):
    q = [(0.0, 0, 0) for _ in range(0)]
    for i, (k, n) in enumerate(cnt.items()):
        if overwrite == 1:
            heapq.heappush(q, (n, i, k))
        else:
            heapq.heappush(q, (n, -i, k))
        if len(q) > cap:
            heapq.heappop(q)
    return [heapq.heappop(q)[2] for _ in range(len(q))][::-1]
   
# save top-k aid2 for each aid1's cnt
@nb.jit(nopython = True, cache = True)
def get_topk(cnts, topk, k):
    for aid1, cnt in cnts.items():
        topk[aid1] = np.array(heap_topk(cnt, 1, k))

In [11]:
topks = {}

# for two modes
for mode in [OP_WEIGHT, TIME_WEIGHT]:
    # get nested counter
    cnts = nb.typed.Dict.empty(
        key_type = nb.types.int64,
        value_type = nb.typeof(nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)))
    max_idx = len(df)
    for idx in tqdm(range(0, max_idx, parallel)):
        row = df.iloc[idx:min(idx + parallel, max_idx)][['session', 'idx', 'length', 'start_time']].values
        get_pairs(aids, ts, ops, row, cnts, ops_weights, mode,params["time_weight"],time_covisitation[mode],params["base_time_weight"])

    # get topk from counter
    topk = nb.typed.Dict.empty(
            key_type = nb.types.int64,
            value_type = nb.types.int64[:])
    get_topk(cnts, topk, topn)

    del cnts; gc.collect()
    topks[mode] = topk
gc.collect()

100%|██████████| 14231/14231 [08:07<00:00, 29.22it/s] 


0

In [12]:
@nb.jit(nopython = True, cache = True)
def inference_(aids, ops, row, result, topk, test_ops_weights, seq_weight, rank_w):
    for session, idx, length in row:
        rank_weight = np.power(2, np.linspace(rank_w, 0, topn))[::-1]
        unique_aids = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
        cnt = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
        cnt_additional_items = nb.typed.Dict.empty(key_type = nb.types.int64, value_type = nb.types.float64)
        
        candidates = aids[idx:idx + length][::-1]
        candidates_ops = ops[idx:idx + length][::-1]
        for a in candidates:
            unique_aids[a] = 0
        sequence_weight = np.power(2, np.linspace(seq_weight, 0, len(candidates)))[::-1]
        for a, op, w in zip(candidates, candidates_ops, sequence_weight):
            if a not in cnt: 
                cnt[a] = 0
            cnt[a] += w * test_ops_weights[op]
            
        if len(unique_aids) >= 20:
            result_candidates = heap_topk(cnt, 0, 20)
        else:
            result_candidates = list(unique_aids)
            for a,w in zip(cnt,sequence_weight):
                if a not in topk: 
                    continue
                for rank, b in enumerate(topk[a]):
                    if b in unique_aids: 
                        continue
                    if b not in cnt_additional_items: 
                        cnt_additional_items[b] = 0
                    cnt_additional_items[b] += w*rank_weight[rank] # add weight equal to the frequency of the weight of the corresponding item
            result_candidates.extend(heap_topk(cnt_additional_items, 0, 20 - len(result_candidates)))
        result[session] = np.array(result_candidates)
        

@nb.jit(nopython = True)
def inference(aids, ops, row, 
              result_clicks, result_buy,result_order,
              topk_clicks, topk_buy,
              test_ops_weights,seq_weight,rank_w):
    inference_(aids, ops, row, result_clicks, topk_clicks, test_ops_weights, seq_weight[0],rank_w[0])
    inference_(aids, ops, row, result_buy, topk_buy, test_ops_weights, seq_weight[1],rank_w[1])
    inference_(aids, ops, row, result_order, topk_buy, test_ops_weights, seq_weight[2],rank_w[2])

In [13]:
result_clicks = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])
result_buy = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])
result_order = nb.typed.Dict.empty(
    key_type = nb.types.int64,
    value_type = nb.types.int64[:])
for idx in tqdm(range(len(df) - len_test, len(df), parallel)):
    row = df.iloc[idx:min(idx + parallel, len(df))][['session', 'idx', 'length']].values
    inference(aids, ops, row, result_clicks, result_buy,result_order, topks[TIME_WEIGHT], topks[OP_WEIGHT], test_ops_weights,params["seq_weight"],params["rank_w"])

100%|██████████| 1633/1633 [01:41<00:00, 16.05it/s]


In [15]:
subs = []
op_names = ["clicks", "carts", "orders"]
for result, op in zip([result_clicks, result_buy, result_order], op_names):

    sub = pd.DataFrame({"session_type": result.keys(), "labels": result.values()})
    sub.session_type = sub.session_type.astype(str) + f"_{op}"
    sub.labels = sub.labels.apply(lambda x: " ".join(x.astype(str)))
    subs.append(sub)
    
sub = pd.concat(subs).reset_index(drop = True)
sub.to_csv(r"C:\Users\mrun7\Downloads\submission.csv", index = False)
sub.head()

,session_type,labels
0,12899779_clicks,59625 737445 731692 1790770 1660529 1253524 19...
1,12899780_clicks,1142000 736515 973453 582732 487136 1502122 88...
2,12899781_clicks,918667 199008 194067 57315 141736 759436 16815...
3,12899782_clicks,834354 595994 740494 987399 889671 779477 1007...
4,12899783_clicks,1817895 607638 1754419 1216820 1729553 300127 ...
